In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/lmei33/Discussion2/master/IBM_HR.csv")
data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
target = "Attrition"
feature_by_dtype = {}
for c in data.columns:
    
    if c == target: continue
    
    data_type = str(data[c].dtype)
    
    if data_type not in feature_by_dtype.keys():
         feature_by_dtype[data_type] = [c]
    else:
        feature_by_dtype[data_type].append(c)

objects = feature_by_dtype["object"]
remove = ["Over18"]
categorical_features = [f for f in objects if f not in remove]
int64s = feature_by_dtype["int64"]
count_features = []
for i in [i for i in int64s if len(data[i].unique()) < 20 and i not in remove]:
    count_features.append(i)
remove.append("StandardHours")
remove.append("EmployeeCount")
count_features += ["TotalWorkingYears", "YearsAtCompany", "HourlyRate"]
remove.append("EmployeeNumber")
numerical_features = [i for i in int64s if i not in remove]
features = categorical_features + numerical_features

for c in categorical_features:
    data[c] = data[c].apply(str)

# Global variables
features, target, categorical_features, numerical_features, count_features
pass

In [4]:
from pandas import get_dummies,concat
onehot_encoded_categorical_data = get_dummies(data[categorical_features])

X = concat([data[numerical_features], onehot_encoded_categorical_data], axis=1)
y = data[target]

In [5]:
def get_results (model, X, y):
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter ("ignore")
        from sklearn.model_selection import cross_val_score
        compute = cross_val_score (model, X, y, cv = 10)
        mean = compute.mean()
        std = compute.std()
        return mean, std
def display_classifier_results (X,y):
    model = []
    
    from sklearn.neighbors import KNeighborsClassifier
    models = [KNeighborsClassifier()]
    
    from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
    model += [GaussianNB(), MultinomialNB(), BernoulliNB()]
    
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier#, VotingClassifier
    models += [RandomForestClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), ExtraTreesClassifier()]
    
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
    models += [LinearDiscriminantAnalysis(), QuadraticDiscriminantAnalysis()]

    from sklearn.svm import SVC, LinearSVC
    models += [SVC(),LinearSVC()]

    from sklearn.linear_model import SGDClassifier
    models += [SGDClassifier()]

    from sklearn.neighbors.nearest_centroid import NearestCentroid
    models += [NearestCentroid()]

    from sklearn.neural_network import MLPClassifier
    models += [MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(30, 2), random_state=1)]

    output = {}

    for m in models:
        try:
            model_name = type(m).__name__
            scores = get_results(m,X,y)
            row = {"Average Score" : scores[0], "Standard Deviation" : scores[1]}
            output[model_name] = row
        except:
            pass

    from pandas import DataFrame
    from IPython.display import display

    display(DataFrame(data=output).T.round(2).sort_values("Average Score", ascending=False))

display_classifier_results(X,y)

,Average Score,Standard Deviation
AdaBoostClassifier,0.88,0.02
LinearDiscriminantAnalysis,0.88,0.02
GradientBoostingClassifier,0.87,0.02
RandomForestClassifier,0.85,0.02
ExtraTreesClassifier,0.85,0.02
SVC,0.84,0.00
MLPClassifier,0.84,0.00
SGDClassifier,0.83,0.03
KNeighborsClassifier,0.82,0.02
LinearSVC,0.73,0.22


With no feature engineering and classifiers on default settings, we can get a Linear Discriminant Analysis classifier or AdaBoostClassifier with 88% accuracy score, just by sklearn's default settings.